### PPGR 2. domaci

## 1. Naivni algoritam

Napisati Python3 funkciju `naivni(origs, imgs)` koja za dve liste od po 4 tačke vraća matricu projektivnog preslikavanja koje prve četiri tačke preslikava redom u druge četiri.

matricu preslikavanja normirati tako da element u donjem desnom uglu bude jednak 1.
matrica treba da bude tipa `np.array` (zato ostavite kod `import numpy as np`, jer vam ta biblioteka treba)
ukoliko tačke `origs` nisu u opštem položaju vratiti string "Losi originali!"
ukoliko tačke `imgs` nisu u opštem položaju vratiti string "Lose slike!"
koristiti `return`, a ne `print` za vraćanje rezultata
kod  `np.set_printoptions(precision=5, suppress=True)`  ostaviti uključen kako bi izlaz bio formatiran na pravni način.

In [24]:
import numpy as np
np.set_printoptions(precision=5, suppress=True)
 
 # ovde pišete pomocne funkcije
def nadji_matricu(pts):
    matrix = np.array([ 
        [pts[0][0], pts[1][0], pts[2][0]],
        [pts[0][1], pts[1][1], pts[2][1]],
        [pts[0][2], pts[1][2], pts[2][2]]
    ])

    D = np.array([pts[3][0], pts[3][1], pts[3][2]])

    result = np.linalg.solve(matrix, D)

    alfa = result[0]
    beta = result[1]
    gama = result[2]

    col1= np.array([alfa*pts[0][0], alfa*pts[0][1], alfa*pts[0][2]])
    col2= np.array([beta*pts[1][0], beta*pts[1][1], beta*pts[1][2]])
    col3= np.array([gama*pts[2][0], gama*pts[2][1], gama*pts[2][2]])

    P = np.column_stack([col1, col2, col3])
    
    return P

def opsti_polozaj(matrix):    
    A = matrix[0]
    B = matrix[1]
    C = matrix[2]
    D = matrix[3]

    m1 = [A, B, C]
    m2 = [A, B, D]
    m3 = [A, C, D]
    m4 = [B, C, D]
    
# determinante
    det1 = np.linalg.det(m1)
    det2 = np.linalg.det(m2)
    det3 = np.linalg.det(m3)
    det4 = np.linalg.det(m4)

    if(det1 > 0 and det1 < 0.000001):
        det1 = 0
    if(det2 > 0 and det2 < 0.000001):
        det2 = 0
    if(det3 > 0 and det3 < 0.000001):
        det3 = 0
    if(det4 > 0 and det4 < 0.000001):
        det4 = 0
    if det1 == 0 or det2 == 0 or det3 == 0 or det4 == 0:
        return False
    return True
    
def naivni(origs, imgs):
 # vaš kod
    # Provera da li su tacke origs i imgs u opstem polozaju
    if not opsti_polozaj(origs):
        return "Losi originali!"
    if not opsti_polozaj(imgs):
        return "Lose slike!"
        
    P1 = nadji_matricu(origs)
    P2 = nadji_matricu(imgs)

    P = np.dot(P2, np.linalg.inv(P1))
    
    # negativna nula
    for i in range(3):
        for j in range(3):
            if(P[i][j] < 0 and P[i][j] > -0.000001):
                P[i][j] = 0
                
    if(P[2][2] != 1 and P[2][2] != 0):
        P = P/P[2][2]
    
    return P

### Test primeri

In [14]:
trapez = [[- 3, - 1, 1], [3, - 1, 1], [1, 1, 1], [- 1, 1, 1]] 
pravougaonik = [[- 2, - 1, 1], [2, - 1, 1], [2, 1, 1], [- 2, 1, 1]]
print(naivni(trapez, pravougaonik))

[[ 1.   0.   0. ]
 [ 0.   1.  -0.5]
 [ 0.  -0.5  1. ]]


In [5]:
origs = [[- 3, - 1, 1], [3, - 1, 1], [1, 1, 1], [- 1, 1, 1]] 
imgs = [[- 2, - 5, 1], [2, - 5, 1], [2, 1, 1], [6, -3, 3]]   #primetite da nisu u opstem polozaju
print(naivni(origs, imgs))

Lose slike!


## 2. DLT Algoritam

Napisati `Python3` funkciju `DLT(origs, imgs)` koja za dve liste od po `n` tačaka vraća matricu projektivnog preslikavanja koje prvih `n` tačaka približno preslikava redom u drugih `n`. Pri tom se koristi **SVD dekompozicija** matrice za odredjivanje optimalnog resenja.

 - matricu preslikavanja normirati tako da element u donjem desnom uglu bude jednak `1`.
 - matrica treba da bude tipa `np.array` (zato ostavite kod `import numpy as np`, jer vam ta biblioteka treba)
 - koristiti `return`, a ne `print` za vraćanje rezultata
 - koristiti `np.linalg.svd(A)`  za singularnu dekompoziciju matrice `A`
 - kod  `np.set_printoptions(precision=5, suppress=True)`  ostaviti uključen kako bi izlaz bio formatiran na pravi način.

In [44]:
import numpy as np
from numpy import linalg  #zbog SVD algoritma
np.set_printoptions(precision=5, suppress=True)
 
 # ovde pišete pomocne funkcije
 
def DLT(origs, imgs):
 # vaš kod
    x1 = origs[0][0]
    x2 = origs[0][1]
    x3 = origs[0][2]

    y1 = imgs[0][0]
    y2 = imgs[0][1]
    y3 = imgs[0][2]

    A = np.array([
        [0, 0, 0, -y3*x1, -y3*x2, -y3*x3, y2*x1, y2*x2, y2*x3],
        [y3*x1, y3*x2, y3*x3, 0, 0, 0, -y1*x1, -y1*x2, -y1*x3]
    ])

    for i in range(1, len(origs)):
        x1 = origs[i][0]
        x2 = origs[i][1]
        x3 = origs[i][2]

        y1 = imgs[i][0]
        y2 = imgs[i][1]
        y3 = imgs[i][2]

        row1 = np.array([0, 0, 0, -y3*x1, -y3*x2, -y3*x3, y2*x1, y2*x2, y2*x3])
        row2 = np.array([y3*x1, y3*x2, y3*x3, 0, 0, 0, -y1*x1, -y1*x2, -y1*x3])

        A = np.vstack((A, row1))
        A = np.vstack((A, row2))
        
# SVD dekompozicija
    U, S, V = np.linalg.svd(A)

    mat = V[-1].reshape(3,3)
    mat = mat/mat[2,2]
    return mat

### Test primeri

In [27]:
trapez = [[- 3, - 1, 1], [3, - 1, 1], [1, 1, 1], [- 1, 1, 1], [1,2,3], [-8,-2,1]] 
pravougaonik1 = [[- 2, - 1, 1], [2, - 1, 1], [2, 1, 1], [- 2, 1, 1], [2,1,5], [-16,-5,5]]
print(DLT(trapez, pravougaonik1))
# ako gledate uradjeni primer, primetite [2,1,4]->[2,1,5], [-16,-5,4]->[-16,-5,5]
# zato matrica nije ista kao u primeru

[[ 0.99611  0.16558 -0.19689]
 [ 0.02513  0.96444 -0.52778]
 [-0.07233 -0.53789  1.     ]]


In [28]:
ulaz = [[891, 490, 48], [584, 778, 139], [845, 108, 707], [910, 714, 827], [572, 917, 660], [747, 417, 313], [388, 347, 217]]
izlaz = [[222, 167, 692], [241, 920, 354], [346, 669, 696], [254, 924, 350], [439, 920, 250], [426, 330, 261], [419, 431, 985]]
print(DLT(ulaz, izlaz))

[[ 2.87711 -3.98074 -0.09904]
 [ 5.24734 -8.44782  0.6669 ]
 [ 3.67964 -4.99412  1.     ]]


## 3. Matrica normalizacije

Napisati `Python3` funkciju `normMatrix(points)` koja za listu od  `n` tačaka ravni (datim homogenim koordinatama) vraća **3x3 matricu normalizacije** tih tačaka. To je 3x3 matrica (kompozicija translacije i homotetije) tako da kada tačke preslikamo tom matricom, dobijamo tačke čije je težište u koordinatnom početku sa prosečnim rastojanjem `sqrt(2)` od koordinatnog početka. 

 - matrica treba da bude tipa `np.array` (zato ostavite kod `import numpy as np`, jer vam ta biblioteka treba)
 - koristiti `return`, a ne `print` za vraćanje rezultata
kod  `np.set_printoptions(precision=5, suppress=True)`  ostaviti uključen kako bi izlaz bio formatiran na pravi način.

In [47]:
import numpy as np
from numpy import linalg
np.set_printoptions(precision=5, suppress=True)
import math
 # ovde pišete pomocne funkcije
 
def normMatrix(points):
 
 # vaš kod
# teziste 
    x = sum([p[0]/p[2] for p in points]) / len(points)
    y = sum([p[1]/p[2] for p in points]) / len(points)
    
# srednje rastojanje
    r = 0.0

    for i in range(len(points)):
        # transl. KP
        t1 = float(points[i][0] / points[i][2]) - x
        t2 = float(points[i][1] / points[i][2]) - y

        r = r + math.sqrt(t1**2 + t2**2)

    r = r / float(len(points))

# skaliranje
    S = float(math.sqrt(2)) / r

    mat = np.array([[S, 0, -S*x], [0, S, -S*y], [0, 0, 1]])
    return mat

### Test primeri

In [37]:
trapez = [[- 3, - 1, 1], [3, - 1, 1], [1, 1, 1], [- 1, 1, 1], [1,2,3], [-8,-2,1]] 
print(normMatrix(trapez))

[[0.44969 0.      0.5746 ]
 [0.      0.44969 0.09993]
 [0.      0.      1.     ]]


In [38]:
pts = [[222, 167, 692], [241, 920, 354], [346, 669, 696], [254, 924, 350], [275, 920, 250], [426, 330, 261], [419, 431, 985]]
print(normMatrix(pts))

[[ 1.1734   0.      -0.90218]
 [ 0.       1.1734  -1.98193]
 [ 0.       0.       1.     ]]


## 4. DLT Algoritam sa normalizacijom

Napisati `Python3` funkciju `DLTwithNormalization(origs, imgs)` koja za dve liste od po `n` tačaka vraća matricu projektivnog preslikavanja koje prvih `n` tačaka približno preslikava redom u drugih `n`. Prvo se  vrši normalizacija originala i slika, a zatim na normalizovane tačke  primenjuje običan **DLT algoritam** (prethodni zadaci).

 - matricu preslikavanja normirati tako da element u donjem desnom uglu bude jednak `1`.
 - matrica treba da bude tipa `np.array` (zato ostavite kod `import numpy as np`, jer vam ta biblioteka treba)
 - koristiti `return`, a ne `prin`t za vraćanje rezultata
 - kod  `np.set_printoptions(precision=5, suppress=True)`  ostaviti uključen kako bi izlaz bio formatiran na pravni način.

In [55]:
import numpy as np
from numpy import linalg  #zbog SVD algoritma
np.set_printoptions(precision=5, suppress=True)
import math
 # ovde pišete pomocne funkcije, a ima ih puno jer koristite sve do sada

def normMatrix(points):
# teziste 
    x = sum([p[0]/p[2] for p in points]) / len(points)
    y = sum([p[1]/p[2] for p in points]) / len(points)
    
# srednje rastojanje
    r = 0.0

    for i in range(len(points)):
        # transl. KP
        t1 = float(points[i][0] / points[i][2]) - x
        t2 = float(points[i][1] / points[i][2]) - y

        r = r + math.sqrt(t1**2 + t2**2)

    r = r / float(len(points))

# skaliranje
    S = float(math.sqrt(2)) / r

    mat = np.array([[S, 0, -S*x], [0, S, -S*y], [0, 0, 1]])
    return mat
    
def DLT(origs, imgs):
    x1 = origs[0][0]
    x2 = origs[0][1]
    x3 = origs[0][2]

    y1 = imgs[0][0]
    y2 = imgs[0][1]
    y3 = imgs[0][2]

    A = np.array([
        [0, 0, 0, -y3*x1, -y3*x2, -y3*x3, y2*x1, y2*x2, y2*x3],
        [y3*x1, y3*x2, y3*x3, 0, 0, 0, -y1*x1, -y1*x2, -y1*x3]
    ])

    for i in range(1, len(origs)):
        x1 = origs[i][0]
        x2 = origs[i][1]
        x3 = origs[i][2]

        y1 = imgs[i][0]
        y2 = imgs[i][1]
        y3 = imgs[i][2]

        row1 = np.array([0, 0, 0, -y3*x1, -y3*x2, -y3*x3, y2*x1, y2*x2, y2*x3])
        row2 = np.array([y3*x1, y3*x2, y3*x3, 0, 0, 0, -y1*x1, -y1*x2, -y1*x3])

        A = np.vstack((A, row1))
        A = np.vstack((A, row2))
        
# SVD dekompozicija
    U, S, V = np.linalg.svd(A)

    mat = V[-1].reshape(3,3)
    mat = mat/mat[2,2]
    return mat

def DLTwithNormalization(origs, imgs):
 
 # vaš kod
 # transformacije
    T = normMatrix(origs)
    T_prim = normMatrix(imgs)

    # normalizovane tacke
    M_line = T.dot(np.transpose(origs))
    M_prim = T_prim.dot(np.transpose(imgs))

    M_line = np.transpose(M_line)
    M_prim = np.transpose(M_prim)

    P_line = DLT(M_line, M_prim)

    mat = (np.linalg.inv(T_prim)).dot(P_line).dot(T)
    
    if(mat[2][2] != 1 and mat[2][2] != 0):
        mat = mat/mat[2][2]

    return mat

### Test primeri

In [52]:
trapez = [[- 3, - 1, 1], [3, - 1, 1], [1, 1, 1], [- 1, 1, 1], [1,2,3], [-8,-2,1]]
pravougaonik1 = [[- 2, - 1, 1], [2, - 1, 1], [2, 1, 1], [- 2, 1, 1], [2,1,5], [-16,-5,5]]
print(DLTwithNormalization(trapez, pravougaonik1))
# ako gledate uradjeni primer, primetite [2,1,4]->[2,1,5], [-16,-5,4]->[-16,-5,5]
# zato matrica nije ista kao u primeru
# stavise matrica se na nekoj decimali razlikuje od one kod DLT algoritma (bez normalizacije)

[[ 0.99732  0.16288 -0.19592]
 [ 0.02538  0.96398 -0.52775]
 [-0.07202 -0.53967  1.     ]]


In [53]:
ulaz = [[891, 490, 48], [584, 778, 139], [845, 108, 707], [910, 714, 827], [572, 917, 660], [747, 417, 313], [388, 347, 217]]
izlaz = [[222, 167, 692], [241, 920, 354], [346, 669, 696], [254, 924, 350], [92, 920, 250], [426, 330, 261], [419, 431, 985]]
print(DLTwithNormalization(ulaz, izlaz))

[[-1.84908  1.02182 -0.61668]
 [-0.61914  0.56214 -4.18024]
 [-5.69947  3.25908  1.     ]]
